In [1]:
!nvidia-smi

Wed Aug 12 11:16:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce 920MX       Off  | 00000000:03:00.0 Off |                  N/A |
| N/A   45C    P0    N/A /  N/A |      5MiB /  2004MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# !git clone https://github.com/ShivamShrirao/dnn_from_scratch.git

In [3]:
# !git clone https://github.com/ShivamShrirao/deep_Q_learning_from_scratch.git

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# %cd deep_Q_learning_from_scratch

In [6]:
import gym
import time
import cv2
import numpy as np

from settings import *
from agent import *
from experience import *
from atari_wrappers import *

In [7]:
agt = Agent(actions=[0,2,3], epsilon=1, min_epsilon=0.1, eps_decay=1e-6)
D_exp = ReplayMemory(capacity=600000)

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
- InputLayer(InputLayer)  (None, 80, 80, 4)          echo             0
__________________________________________________________________________________________
0 Conv2D(Conv2D)          (None, 40, 40, 32)         relu             1184
__________________________________________________________________________________________
1 Conv2D(Conv2D)          (None, 20, 20, 64)         relu             18496
__________________________________________________________________________________________
2 Conv2D(Conv2D)          (None, 10, 10, 128)        relu             73856
__________________________________________________________________________________________
3 Flatten(Flatten)        (None, 12800)              echo             0
__________________________________________________________________________________________
4 Den

In [8]:
env = gym.make('Pong-v0')
env = FrameStack(env, NFRAMES)      # preprocess and stack frames

In [9]:
len(D_exp.ndone)

0

In [14]:
agt.model.load_weights("model.w8s")

In [15]:
agt.epsilon = 0

In [16]:
for i_episode in range(3):
    observation = env.reset()
    ep_score = 0
    start = time.time()
    sv_obs = deque([], maxlen=NFRAMES)
    sv_act = deque([], maxlen=NFRAMES)
    sv_rew = deque([], maxlen=NFRAMES)
    sv_don = deque([], maxlen=NFRAMES)
    for t in range(10000):
        env.render()
        action = agt.get_action(observation)
        next_observation, reward, done, info = env.step(action)
        ep_score += reward

        sv_obs.append(observation)
        sv_act.append(action)
        sv_rew.append(reward)
        sv_don.append(done)

#         D_exp.store_transition(sv_obs[0], agt.actions.index(sv_act[0]), sv_rew[0], next_observation, sv_don[0])
        observation = next_observation
#         time.sleep(1/80)

        # D_exp.current_state.popleft()
        # D_exp.action_idx.popleft()
        # D_exp.reward.popleft()
        # D_exp.next_state.popleft()
        # D_exp.ndone.popleft()

#         if len(D_exp.ndone) > BATCH_SIZE:
#             agt.train(D_exp, BATCH_SIZE)
        
        if done:
            break
        print('\r', t, action, ep_score, end='  ')
    print(f"\rEpisode {i_episode+1} finished after {t+1} timesteps, Score: {ep_score}, Epsilon: {agt.epsilon}, Time: {time.time()-start}")
#     agt.model.save_weights("/content/drive/My Drive/model.w8s")
env.close()

Episode 1 finished after 1024 timesteps, Score: -21.0, Epsilon: 0, Time: 5.9434897899627686
Episode 2 finished after 1016 timesteps, Score: -21.0, Epsilon: 0, Time: 5.845656156539917
Episode 3 finished after 1019 timesteps, Score: -21.0, Epsilon: 0, Time: 5.788917064666748


In [13]:
env.close()